# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [1]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [3]:
# Dataset before augmentation
# data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
#data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)
data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)


# Dataset after augmentation
data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)


# Check Shapes of Datasets (they must be the same)

In [7]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")


Shape of data1: (172, 13)
Shape of data2: (240, 25)


In [9]:
# If the datasets do not have the same columns ---> make one dataset match the other one (order does not matter)
# Subset columns so both have the same 
# (we need to make sure we have the same categorical and numerical columns) ---> you do this after this cell
data2 = data2.iloc[:, -13:]
data2.shape

(240, 13)

# Convert columns into Categorical (Only for Uniform and Gaussian Distributions)

In [11]:
for i in range(-5,0):
    data1.iloc[:,i] = data1.iloc[:,i].astype(int) # Integer
    data1.iloc[:,i] = data1.iloc[:,i].astype('category') # Categories
data1.dtypes 

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8     category
9     category
10    category
11    category
12    category
dtype: object

In [ ]:
for i in range(-5,0):
    data2.iloc[:,i] = data2.iloc[:,i].astype(int) # Integer
    data2.iloc[:,i] = data2.iloc[:,i].astype('category') # Categories
data2.dtypes
#data2.head()  

# Convert columns into Categorical (Only for Stacked Distribution)

In [ ]:
for column in data1.columns[-9:]:
    data1[column] = data1[column].astype('category')

# Verify the changes
print(data1.dtypes)


In [20]:
for column in data2.columns[-5:]:
    data2[column] = data2[column].astype('category')

# Verify the changes
print(data2.dtypes)


12     float64
13     float64
14     float64
15     float64
16     float64
17     float64
18     float64
19     float64
20    category
21    category
22    category
23    category
24    category
dtype: object


# Data1: Synthetic Data

In [13]:
# data2
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])


##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)



#################### save Data1 matrix as a dataframe #################################
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

# Data1 + Data2: Combined_data

In [25]:
########################## Concat Data1 +  Data2 ##################################
# Concat (adding rows)
combined_df = np.vstack((data1, data2))

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data1.columns)

# Convert the last number of columns to categorical (to make sure they have the same)
for column in combined_data.columns[-5:]:
    combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)


##################### Split Combined_data ################################

# Split dataset into X and y (data1)
X = combined_data.iloc[:, :-1]  # All columns except the last one
y = combined_data.iloc[:, -1]   # Only the last column


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])


##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)


############################## save combined_data matrix as a dataframe ############################
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape


# Differences in Matrices (Frobenius Norm)

In [32]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_abs = np.linalg.norm(matrix1 - matrix2, ord='fro')   # Absolute error with Frobenius norm

frobenius_rel = frobenius_abs/np.linalg.norm(matrix1, ord='fro')    # Relative error with Frobenius norm

print(f"Frobenius norm (absolute error) : {frobenius_abs: .3f}")
print(f"Frobenius norm (relative error) : {frobenius_rel: .3f}")


Frobenius norm (absolute error) :  5.313
Frobenius norm (relative error) :  1.669


# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
